# Chapter 15. Missing Data and Other Opportunities

### Code 15.1

### Code 15.2

### Code 15.3

### Code 15.4

### Code 15.5

### Code 15.6

### Code 15.7

### Code 15.8

### Code 15.9

### Code 15.10

### Code 15.11

### Code 15.12

### Code 15.13

### Code 15.14

### Code 15.15

### Code 15.16

### Code 15.17

### Code 15.18

### Code 15.19

### Code 15.20

### Code 15.21

### Code 15